In [34]:
#Parameters common to all units (known)
r = Float32(0.05);
delta = Float32(0.15);
lambda = Float32(0.015);
v = Float32(0.15);
etaA = Float32(-0.1);
etaP = Float32(0.10);
kappaH = log(3);
kappaL = -log(3);

k = [r, delta, lambda, v, etaA, etaP, kappaH, kappaL];


In [35]:
#Parameters common to all units (unknown)
ksiH = Float32(0.1);
ksiL = Float32(0.9);

ksi = [ksiH, ksiL];

In [4]:
# Parameters specific to each unit (known)
mu = Float32(0.0729);
alpha = Float32(0.10);
sigmaP = Float32(0.7559);
sigmaA = Float32(0.1280);
rho =  Float32(-0.0370);
chiA =  Float32(-0.0077);
chiP = Float32(0.2017);


k_g = [mu, alpha, sigmaP, sigmaA, rho, chiA,chiP];

7-element Vector{Float32}:
  0.0729
  0.1
  0.7559
  0.128
 -0.037
 -0.0077
  0.2017

In [36]:
# Parameters specific to each unit (unknown)
theta = Float32(1.80);
gamma = Float32(0.06);
phiH = Float32(0.001);
phiL = Float32(0.50);
lH = Float32(0.8);
lL = Float32(0.4);

Kappa_g = [theta, gamma, phiH, phiL, lH, lL];

In [6]:
# Initial guess for the unknown boundaries
# x0 = [x_lH m_H m_L x_uH x_uL]

x0 = [0.02 0.15 0.20 0.36 0.41];


In [7]:
mutable struct params
    r::Float64;delta::Float64;v::Float64;sigmaP::Float64;sigmaA::Float64;theta::Float64;lambda::Float64;
    l::Float64;gamma::Float64;phi::Float64;ksi::Float64;etaA::Float64;etaP::Float64;chiA::Float64;chiP::Float64;
    rho::Float64;alpha::Float64;mu::Float64
end


In [37]:
function parameters(k, ksi, k_g, Kappa_g)
    #Create parameter structures for each state s = H, L
    paramsH =  params(k[1],k[2],k[4],k_g[3],k_g[4],Kappa_g[1],k[3],Kappa_g[5],Kappa_g[2],Kappa_g[3],
        exp(k[7])*ksi[1],k[5],k[6],k_g[6],k_g[7],k_g[5],k_g[2] - sigmaA * etaA * chiA,k_g[1] - sigmaP *etaP * chiP)

    paramsL = params(k[1],k[2],k[4],k_g[3],k_g[4],Kappa_g[1],k[3],Kappa_g[6],Kappa_g[2],Kappa_g[4],
        exp(k[8]) * ksi[2],k[5],k[6],k_g[6],k_g[7],k_g[5],k_g[2] - sigmaA * etaA * chiA,k_g[1] - sigmaP * etaP * chiP)

    return [paramsL paramsH]
end


parameters (generic function with 1 method)

In [38]:
function twoode(c, f, region, paramsH, paramsL)
    # ODE equations
    a13 = 0.5 * (paramsH.sigmaP^2 * c^2 - 2 * paramsH.sigmaP *paramsH.sigmaA * paramsH.rho * c + paramsH.sigmaA^2)
    a23 = 0.5 * (paramsL.sigmaP^2 * c^2 - 2 * paramsL.sigmaP *paramsL.sigmaA * paramsL.rho * c + paramsL.sigmaA^2)
    i1 = (1 / paramsH.theta) * ((f[1] / f[2]) - c - 1) + paramsH.v
    g_i1 = 0.5 * paramsH.theta * (i1 - paramsH.v)^2

    i2 = (1 / paramsL.theta) * ((f[3] / f[4]) - c - 1) + paramsL.v
    g_i2 = 0.5 * paramsL.theta * (i2 - paramsL.v)^2

    a2 = (paramsH.r * f[1] - ((paramsH.r - paramsH.lambda) * c +
        paramsH.alpha - i1 - g_i1) * f[2] - (i1 - paramsH.delta
        + paramsH.mu) * (f[1] - c * f[2]) - paramsH.ksi *
        (f[3] - f[1])) / a13

    a4 = (paramsL.r * f[3] - ((paramsL.r - paramsL.lambda) * c +
        paramsL.alpha - i2 - g_i2) * f[4] - (i2 - paramsL.delta
        + paramsL.mu) * (f[3] - c * f[4]) - paramsL.ksi *
        (f[1] - f[3])) / a23

    dfdc = [f[2] a2 f[4] a4]

    return dfdc
end

twoode (generic function with 1 method)

In [39]:
function twobc(yl, yr, p, paramsH, paramsL)
    # Boundary conditions
    i1 = (1 / paramsH.theta) * (yr[1,4] - p[4] - 1) + paramsH.v;
    g_i1 = 0.5 * paramsH.theta * (i1 - paramsH.v)^2;

    i2 = (1 / paramsL.theta) * (yr[3,5] - p[5] - 1) + paramsL.v;
    g_i2 = 0.5 * paramsL.theta * (i2 - paramsL.v)^2;

    res = [
            yl[3,1] - yr[3,3] + paramsL.phi +
            (1 + paramsL.gamma) * p[3],

            yl[1,2] - yr[1,2] + paramsH.phi +
            (1 + paramsH.gamma) * (p[2] - p[1]),

            yl[1,2] - yr[1,1],
            yl[2,2] - yr[2,1],
            yl[3,2] - yr[3,1],
            yl[4,2] - yr[4,1],

            # Conditions at m_H
            yl[1,3] - yr[1,2],
            yl[2,3] - yr[2,2],
            yl[3,3] - yr[3,2],
            yl[4,3] - yr[4,2],

            # Conditions at m_L
            yl[1,4] - yr[1,3],
            yl[2,4] - yr[2,3],
            yl[3,4] - yr[3,3],
            yl[4,4] - yr[4,3],

            # Conditions at upper boundary 1
            yl[2,5] - yr[2,4],
            yl[3,5] - yr[3,4],
            yl[4,5] - yr[4,4],

            # Conditions at upper boundary 2
            yr[1,5] - yl[1,5] - p[5] + p[4],

            paramsH.r * yr[1,4]- (i1 - paramsH.delta + paramsH.mu)*(yr[1,4] - p[4])-((paramsH.r - paramsH.lambda) * p[4] +paramsH.alpha - i1 - g_i1)- paramsH.ksi *(yr[3,4] - yr[1,4]),
            paramsL.r * yr[3,5] - (i2 - paramsL.delta + paramsL.mu) *(yr[3,5] - p[5])- ((paramsL.r - paramsL.lambda) * p[5] +paramsL.alpha - i2 - g_i2) - paramsL.ksi * (yr[1,5] - yr[3,5])
            ];
end


twobc (generic function with 1 method)

In [11]:
parameters(k, ksi, k_g, Kappa_g)

1×2 Matrix{params}:
 params(0.05, 0.15, 0.15, 0.7559, 0.128, 1.8, 0.015, 0.4, 0.06, 0.5, 0.3, -0.1, 0.1, -0.0077, 0.2017, -0.037, 0.0999014, 0.0576535)  …  params(0.05, 0.15, 0.15, 0.7559, 0.128, 1.8, 0.015, 0.8, 0.06, 0.001, 0.3, -0.1, 0.1, -0.0077, 0.2017, -0.037, 0.0999014, 0.0576535)

In [12]:
#using Pkg; Pkg.add("ODEInterface")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed FFTW_jll ─ v3.3.9+8
  No Changes to `~/.julia/environments/v1.6/Project.toml`
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [f5851436] ↑ FFTW_jll v3.3.9+7 ⇒ v3.3.9+8
Precompiling project...
  ✓ FFTW_jll
  ✓ FFTW
  ✓ KernelDensity
  ✓ Gadfly
4 dependencies successfully precompiled in 23 seconds (70 already precompiled)


In [40]:
"""
  Module for testing ODEInterface
  """
module ODEInterfaceTest

using Test

using ODEInterface
@ODEInterface.import_huge

const solvers = (odex, odex_i32)

const solvers_colnew  = ( colnew, colnew_i32 )


"""
  create a callable-type in order to check, if solvers
  can handle callable-types (which are not a subclass of
  Function) as right-hand sides.
  """
mutable struct Callable_Type
  param :: Float64
end

function (ct::Callable_Type)(t,x)
  return ct.param*x
end

function mylinspace(a, b, length::Integer)
  return collect(range(a, stop=b, length=length))
end



function test_colnew1(solver::Function)
  a, b = -pi/2, pi/2 # x-tremes
  orders = [1, 1,] # orders of equation
  ζ = [a, b] # x-interval
 
  ε = nothing  # parameter
  ε_old = nothing # previous value of parameter
  sol_old = nothing # previous step solution

  function rhs(x, z, f)
      s² = sin(x)^2 # 
      f[1] = (s²-z[2]*s²*s²/z[1])/ε
      f[2] = 0.0
  end
  
  function Drhs(x, z, df)
      df[:] .= 0.0
      s⁴ = sin(x)^4
      df[1,1] = z[2]*s⁴/(z[1]^2)
      df[1,2] = -s⁴/z[1]
  end
  
  function bc(i, z, bc)
      bc[1] = z[1]-1.0
  end
  
  function Dbc(i, z, dbc)
      dbc[1] = 1.0
      dbc[2] = 0.0
  end
  
  function initial_guess(x, z, dmz)
      z[1] = 0.5
      z[2] = 1.0
      rhs(x, z, dmz)
  end

  opt = OptionsODE("colnew1",
        OPT_BVPCLASS => 2, OPT_COLLOCATIONPTS => 7,
        OPT_RTOL => [1e-4, 1e-4], OPT_MAXSUBINTERVALS => 200)

  sol = nothing
  for ε_new = [1.0, 0.5, 0.2, 0.1]
    ε = ε_new
    guess = sol_old !== nothing ? sol_old : initial_guess    
    sol, retcode, stats = colnew([a,b], orders, ζ, rhs, Drhs, bc, Dbc, guess ,opt);
    @assert retcode>0
    sol_old = sol; ε_old = ε
  end
  
  z₀ = evalSolution(sol, 0.0)
  @assert isapprox(z₀[1], 0.161671, rtol=1e-3,atol=1e-3)
  @assert isapprox(z₀[2], 1.01863, rtol=1e-3,atol=1e-3)
  return true
end

function test_colnew()
  problems = (test_colnew1, )
  @testset "colnew" begin
    @testset for solver in solvers_colnew,
                  problem in problems
      @test problem(solver)
    end
  end
end


function test_all()
  test_colnew()
end

test_all()

end

# vim:syn=julia:cc=79:fdm=indent:


Test Summary: | Pass  Total
colnew        |    2      2


Main.ODEInterfaceTest

In [41]:
b = [0.0485 0.1184 0.2597 0.4679 0.6719];
parms = parameters(k, ksi, k_g, Kappa_g);
parms[1], parms[2]

(params(0.05000000074505806, 0.15000000596046448, 0.15000000596046448, 0.7559000253677368, 0.12800000607967377, 1.7999999523162842, 0.014999999664723873, 0.4000000059604645, 0.05999999865889549, 0.5, 0.299999992052714, -0.10000000149011612, 0.10000000149011612, -0.007699999958276749, 0.20170000195503235, -0.03700000047683716, 0.09990143775939941, 0.05765349417924881), params(0.05000000074505806, 0.15000000596046448, 0.15000000596046448, 0.7559000253677368, 0.12800000607967377, 1.7999999523162842, 0.014999999664723873, 0.800000011920929, 0.05999999865889549, 0.0010000000474974513, 0.3000000044703484, -0.10000000149011612, 0.10000000149011612, -0.007699999958276749, 0.20170000195503235, -0.03700000047683716, 0.09990143775939941, 0.05765349417924881))

In [21]:
f_init = [1 1 1 1];
x_init = [0 b[1] b[1] b[2] b[2] b[3] b[3] b[4] b[4] b[5] ];


In [31]:
#  dummy check function ODE
c = 0.1
f = [1 2 3 4]
region = 1
twoode(c, f, region, parms[1], parms[2])

1×4 Matrix{Float64}:
 2.0  -61.6227  4.0  57.0166

In [33]:

# dummy check function residuals
yl = [0.5538    0.5972    0.6723    0.8176    1.0246;
    0.5667    1.0599    1.0600    1.0095    1.0000;
    0.0339    0.4622    0.6386    0.8092    1.0219;
   16.1030    4.4192    1.5753    1.0600    1.0042]

yr = [0.5972    0.6723    0.8176    1.0263    1.2286;
    1.0599    1.0600    1.0095    1.0000    1.0008;
    0.4622    0.6386    0.8092    1.0219    1.2262;
    4.4192    1.5753    1.0600    1.0042    1.0000]

bc = twobc(yl, yr, b, parms[1], parms[2])


20-element Vector{Float64}:
 -0.4990180003007874
  0.49899399990625676
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.1102230246251565e-16
 -6.244159435178035e-6
  3.4598981606691055e-5

The only job is now to debug the COLNEW Julia interface and pipeline it to the current code